In [1]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import re
import os


In [2]:
html_doc = source = urllib.request.urlopen('https://www.newsguardtech.com/coronavirus-misinformation-tracking-center/').read()
soup = BeautifulSoup(html_doc, 'html.parser')

In [3]:
fakelist = [x.string.lower() for x in soup.find_all('a') if x.string is not None and len(re.findall('\\.',x.string)) > 0][1:]
#re.findall('.*\\.',x.string)[0][:-1]
print(len(fakelist))
fakelist

290


['activistpost.com',
 'ageofautism.com',
 'americanthinker.com',
 'bb4sp.com',
 'beforeitsnews.com',
 'bigleaguepolitics.com',
 'breakingchristiannews.com',
 'brighteon.com',
 'buffalochronicle.com',
 'channel22news.com',
 'childrenshealthdefense.org',
 'clashdaily.com',
 'collective-evolution.com',
 'conservativedailypost.com',
 'conservativepost.com',
 'conservativetreehouse.com',
 'curiousmindmagazine.com\xa0',
 'dcclothesline.com',
 'dcdirtylaundry.com',
 'diamondandsilk.com',
 'doctordavidfriedman.com',
 'drsergegregoire.com',
 'en-volve.com',
 'fakepandemic.com',
 'frontpagemag.com',
 'ftbagency.com',
 'gellerreport.com',
 'glennbeck.com',
 'gnews.org',
 'greenmedinfo.com',
 'healingoracle.ch\xa0',
 'healthimpactnews.com',
 'healthnutnews.com',
 'herbs-info.com',
 'holistichealth.one',
 'homenaturalcures.com',
 'humansarefree.com',
 'informationliberation.com',
 'infowars.com',
 'intellihub.com',
 'jimbakkershow.com',
 'jimhumble.co',
 'madworldnews.com',
 'medicine-today.net',
 

In [4]:
cwd = os.getcwd()
cwd

'/home/j0hndoe/Documents/git/reddit-disinformation/disinfo'

In [7]:
subm_files = sorted(re.findall('SUBM_2020_[0-9][0-6].csv', ' '.join(os.listdir(cwd+'/../output'))))
comm_files = sorted(re.findall('COMM_2020_[0-9][0-6].csv', ' '.join(os.listdir(cwd+'/../output'))))
match_files = sorted(re.findall('MATCH_2020_[0-9][0-6].csv', ' '.join(os.listdir(cwd+'/../output'))))

In [8]:
SS = pd.DataFrame()
for ss,cc,mm in zip(subm_files, comm_files, match_files):
    SUBM = pd.read_csv(cwd+'/../output/'+ss, lineterminator = '\n')
    COMM = pd.read_csv(cwd+'/../output/'+cc, lineterminator = '\n')
    MATCH = pd.read_csv(cwd+'/../output/'+mm, lineterminator = '\n')
    SUBM['was_removed'] = (~SUBM.removed_by_category.isna()).astype(int)
    FLAGS = COMM.rename(columns={'id':'comm_id','parent_id':'subm_id'})\
                    [['subm_id','comm_id']].set_index('comm_id').\
                    join(MATCH[['comm_id','flag']].\
                    set_index('comm_id')).fillna(0).\
                    groupby('subm_id')['flag'].agg('sum')
    SS0 = SUBM.rename(columns={'id':'subm_id'})[['subm_id','subreddit','domain','was_removed']].\
                set_index('subm_id').join(FLAGS, how = 'left').fillna(0)
    SS=SS.append(SS0)
    print('Finished: ' + ss)

Finished: SUBM_2020_01.csv
Finished: SUBM_2020_02.csv
Finished: SUBM_2020_03.csv


/home/j0hndoe/anaconda3/envs/jupyter-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Finished: SUBM_2020_04.csv
Finished: SUBM_2020_05.csv
Finished: SUBM_2020_06.csv


In [10]:
SS['domain'] = SS.domain.astype('str')
SS['isflagged'] = (SS.flag>0).astype('int')
SS['fakedomain'] = (SS.domain.isin(fakelist)).astype('int')
SS['total']=1
SS

,subreddit,domain,was_removed,flag,isflagged,fakedomain,total
subm_id,,,,,,,
t3_ewwm6v,conspiracy,self.conspiracy,0,0.0,0,0,1
t3_ewwm6n,WayOfTheBern,self.WayOfTheBern,0,0.0,0,0,1
t3_ewwm65,Coronavirus,self.Coronavirus,0,0.0,0,0,1
t3_ewwm5r,unitedkingdom,self.unitedkingdom,0,0.0,0,0,1
t3_ewwm4u,onguardforthee,cbc.ca,0,0.0,0,0,1
...,...,...,...,...,...,...,...
t3_gu9y7i,socialism,self.socialism,1,0.0,0,0,1
t3_gu9y6h,newzealand,self.newzealand,0,0.0,0,0,1
t3_gu9y6z,news,nytimes.com,1,0.0,0,0,1


In [12]:
#SS.domain.sample(20)
SSD = SS[~SS.domain.str.contains('redd\\.it|^self\\.|twitter|/r/|youtu|img|gif|giph|blog')]
SSD.reset_index().to_csv("SSD.csv")
SSD

,subreddit,domain,was_removed,flag,isflagged,fakedomain,total
subm_id,,,,,,,
t3_ewwm4u,onguardforthee,cbc.ca,0,0.0,0,0,1
t3_ewwm3k,Conservative,foxnews.com,0,0.0,0,0,1
t3_ewwm2o,TechNewsToday,cnet.com,0,0.0,0,0,1
t3_ewwm2a,nottheonion,digitaltrends.com,1,1.0,1,0,1
t3_ewwm0t,worldnews,news.ycombinator.com,0,0.0,0,0,1
...,...,...,...,...,...,...,...
t3_gu9yjj,news,twitch.tv,1,0.0,0,0,1
t3_gu9yfb,nyc,thepetitionsite.com,1,0.0,0,0,1
t3_gu9yeu,nottheonion,livescience.com,1,1.0,1,0,1


In [13]:
pdc = pd.crosstab(SSD.isflagged, SSD.fakedomain)
pdc

fakedomain,0,1
isflagged,,
0,1764376,22246
1,51445,830


In [14]:
round(100*pdc/pdc.agg('sum'),3)

fakedomain,0,1
isflagged,,
0,97.167,96.403
1,2.833,3.597


In [15]:
import numpy as np
round(100*(pdc.transpose()/pdc.transpose().agg('sum')).transpose(),3)

fakedomain,0,1
isflagged,,
0,98.755,1.245
1,98.412,1.588


### Aggregate at domain level

In [36]:
SSD = pd.read_csv("SSD.csv")

In [16]:
SSD.domain = SSD.domain.astype('str')

/home/j0hndoe/anaconda3/envs/jupyter-env/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [37]:
DOMAGG = SSD[SSD.fakedomain==0].groupby('domain')[['isflagged','total']].agg('sum')
DOMAGG['pct'] = round(100*(DOMAGG.isflagged/DOMAGG.total),2)
DOMAGG

,isflagged,total,pct
domain,,,
-coronavirus-cases-are-mild-complicating-efforts-to-respond,0,1,0.00
0,9,455,1.98
000webhost.com,0,1,0.00
007creditagent.mystrikingly.com,0,1,0.00
007store.com,0,1,0.00
...,...,...,...
zybrew.beer,0,2,0.00
zycrypto.com,0,1,0.00
zytebrand.wixsite.com,0,1,0.00


Fake news websites

In [38]:
pd.set_option('display.max_rows', 200)
DOMAGGF = SSD[SSD.fakedomain>0].groupby('domain')[['isflagged','total']].agg('sum')
DOMAGGF['pct'] = round(100*(DOMAGGF.isflagged/DOMAGGF.total),2)
print(DOMAGGF.sort_values('total', ascending = False))
pd.set_option('display.max_rows', 30)

                              isflagged  total     pct
domain                                                
greenmedinfo.com                      1   4832    0.02
redstate.com                         89   3173    2.80
thegatewaypundit.com                135   3076    4.39
zerohedge.com                       161   2975    5.41
pjmedia.com                          71   1573    4.51
americanthinker.com                  26   1396    1.86
theepochtimes.com                    56    882    6.35
frontpagemag.com                     14    580    2.41
globalresearch.ca                    47    512    9.18
theconservativetreehouse.com          5    453    1.10
wnd.com                              16    326    4.91
infowars.com                         18    303    5.94
oann.com                             16    229    6.99
presstv.com                           3    209    1.44
gellerreport.com                      6    206    2.91
truepundit.com                        5    146    3.42
thedonald.

In [39]:
fakelist_mins = list(DOMAGGF[DOMAGGF.total>=20].index)
nonfakelist_mins = list(DOMAGG[DOMAGG.total>=20].index)

In [22]:
print('%7.0d / %7.0d (%5.2f%%) fake news domains present on Reddit.' %
      (len(set(SSD.domain.unique()).intersection(set(fakelist))),
       len(set(fakelist)),
       100*len(set(SSD.domain.unique()).intersection(set(fakelist)))/len(set(fakelist))))

print('\n%7.0d / %7.0d (%5.2f%%) fake news domains flagged on Reddit.' %
      (len(set(SSD.domain[SSD.isflagged>0].unique()).intersection(set(fakelist))),
       len(set(SSD.domain.unique()).intersection(set(fakelist))),
       100*len(set(SSD.domain[SSD.isflagged>0].unique()).intersection(set(fakelist))) / len(set(SSD.domain.unique()).intersection(set(fakelist)))))

print('%7.0d / %7.0d (%5.2f%%) non-fake news domains flagged on Reddit.' %
      (len(set(SSD.domain[SSD.isflagged>0].unique()).difference(set(fakelist))),
       len(set(SSD.domain.unique()).difference(set(fakelist))),
       100*len(set(SSD.domain[SSD.isflagged>0].unique()).difference(set(fakelist))) / len(set(SSD.domain.unique()).difference(set(fakelist)))))

print('\n%7.0d / %7.0d (%5.2f%%) fake news domains (with at least 5 submissions) flagged on Reddit.' %
      (len(set(SSD.domain[SSD.isflagged>0].unique()).intersection(set(fakelist_mins))),
       len(set(SSD.domain.unique()).intersection(set(fakelist_mins))),
       100*len(set(SSD.domain[SSD.isflagged>0].unique()).intersection(set(fakelist_mins))) / len(set(SSD.domain.unique()).intersection(set(fakelist_mins)))))

print('%7.0d / %7.0d (%5.2f%%) non-fake news domains (with at least 5 submissions) flagged on Reddit.' %
      (len(set(SSD.domain[SSD.isflagged>0].unique()).intersection(set(nonfakelist_mins))),
       len(set(SSD.domain.unique()).intersection(set(nonfakelist_mins))),
       100*len(set(SSD.domain[SSD.isflagged>0].unique()).intersection(set(nonfakelist_mins))) / len(set(SSD.domain.unique()).intersection(set(nonfakelist_mins)))))

    110 /     290 (37.93%) fake news domains present on Reddit.

     57 /     110 (51.82%) fake news domains flagged on Reddit.
   5859 /   82904 ( 7.07%) non-fake news domains flagged on Reddit.

     44 /      52 (84.62%) fake news domains (with at least 5 submissions) flagged on Reddit.
   2947 /    5341 (55.18%) non-fake news domains (with at least 5 submissions) flagged on Reddit.


In [23]:
DOMAGGF[DOMAGGF.pct>0].pct.mean()

14.044736842105262

In [24]:
DOMAGG[DOMAGG.pct>0].pct.mean()

21.479249018603856

In [26]:
SSDSUBR = SSD[SSD.fakedomain>0].groupby(['subreddit','domain']).\
                agg(totalposts = ('total','sum'), totalflagged = ('isflagged','sum')).\
                assign(ispresent = lambda df: (df.totalposts>0).astype('int'),
                       wasflagged = lambda df: (df.totalflagged>0).astype('int'))
SSDSUBR.head()

totalposts  totalflagged  \
subreddit        domain                                               
AmericanPolitics americanthinker.com               13             0   
                 bigleaguepolitics.com              1             0   
                 brighteon.com                      1             0   
                 clashdaily.com                     1             0   
                 collective-evolution.com           1             0   

                                           ispresent  wasflagged  
subreddit        domain                                           
AmericanPolitics americanthinker.com               1           0  
                 bigleaguepolitics.com             1           0  
                 brighteon.com                     1           0  
                 clashdaily.com                    1           0  
                 collective-evolution.com          1           0

### Which subreddits post and flag dubious sources

In [29]:
pd.set_option('display.max_rows', 200)
print(SSDSUBR.reset_index().\
        groupby('subreddit')[['ispresent','wasflagged']].agg('sum').\
        assign(percflagged = lambda df: round(100*df.wasflagged/df.ispresent,2)))
pd.set_option('display.max_rows', 30)

                       ispresent  wasflagged  percflagged
subreddit                                                
AmericanPolitics              22           5        22.73
BreadTube                      1           0         0.00
COVID19                       18           2        11.11
CanadaCoronavirus              1           1       100.00
CanadaPolitics                 4           0         0.00
ChapoTrapHouse                12           1         8.33
China                         18           2        11.11
China_Flu                     37          14        37.84
Conservative                  49          17        34.69
CoronaVirus_2019_nCoV          4           0         0.00
Coronavirus                   62          29        46.77
CoronavirusCA                  3           1        33.33
CoronavirusDownunder           4           0         0.00
CoronavirusMemes               1           0         0.00
CoronavirusRecession           1           1       100.00
CoronavirusUK 

### Do flagged comments get removed?

In [30]:
pd.crosstab(SSD.was_removed, SSD.isflagged)

isflagged,0,1
was_removed,,
0,1027094,40733
1,759528,11542


In [31]:
pd.crosstab(SSD.was_removed, SSD.isflagged).apply(lambda r: r/r.sum(), axis=0)

isflagged,0,1
was_removed,,
0,0.57488,0.779206
1,0.42512,0.220794


In [32]:
SSD['fndom'] = SSD.domain.isin(fakelist).astype(int)
SSD.groupby('fndom')['was_removed'].agg(['mean'])

<ipython-input-32-374ccb4d9171>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SSD['fndom'] = SSD.domain.isin(fakelist).astype(int)


,mean
fndom,
0,0.420318
1,0.340094


In [34]:
SSDOM2 = SSD.groupby('domain')['was_removed'].agg(['sum','count']).reset_index()
SSDOM2['percremoved'] = SSDOM2['sum']/SSDOM2['count']
SSDOM2['fndom'] = SSDOM2.domain.isin(fakelist).astype(int)

In [35]:
SSDOM2.groupby('fndom')['percremoved'].agg(['min','max','mean','median'])

,min,max,mean,median
fndom,,,,
0,0.0,1.0,0.546128,0.625000
1,0.0,1.0,0.315165,0.195021
